In [1]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 4.4 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=b09c9d31dbabed2a84276420af753d02ed8edc393e569865d99997d652df15a6
  Stored in directory: /home/vscode/.cache/pip/wheels/d4/c8/5b/b5be9c20e5e4503d04a6eac8a3cd5c2393505c29f02bea0960
Successfully built bs4

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import requests
from bs4 import BeautifulSoup
import csv

# Leer las URL desde el archivo CSV
url_list = []

with open('url_products.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        url = row['URL']
        url_list.append(url)

# Crear una lista para almacenar los datos de cada producto
all_product_data = []

for url in url_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Acceder a los atributos 'data-*' de la etiqueta 'section'
    product_section = soup.select_one('section.c-product')

    product_id = product_section['data-id']
    product_sku = product_section['data-sku']
    product_name = product_section['data-name']
    product_brand = product_section['data-brand']
    product_category = product_section['data-category']
    product_variant = product_section['data-variant']
    product_price = product_section['data-price']

    # Acceder a la ficha técnica
    wine_table = soup.find('table', class_='table table-striped')

    # Inicializar variables para los campos de la tabla
    wine_data = {
        "Viña": "",
        "Cepa": "",
        "Cosecha": "",
        "Procedencia": "",
        "Línea": "",
        "Formato": "",
        "Tipo": "",
        "Alcohol": "",
        "Mezcla": "",
        "Guarda": "",
        "Maridaje": ""
    }

    if wine_table:
        # Obtener todas las filas de la tabla
        rows = wine_table.find_all('tr')

        for row in rows:
            header = row.find('th')
            data = row.find('td')

            if header and data:
                header_text = header.text.strip()
                data_text = data.text.strip()

                # Asignar los datos a las categorías correspondientes
                if header_text in wine_data:
                    wine_data[header_text] = data_text

    # Acceder al puntaje y calificaciones
    score_section = product_section.select_one('.c-product__score')

    if score_section is not None:
        lacav_score_element = score_section.select_one('.o-score--lacav .o-score__points')
        if lacav_score_element is not None:
            lacav_score = lacav_score_element.text
        else:
            lacav_score = "No score available"
    else:
        lacav_score = "No score available"

    # Descripción
    description = soup.select_one('.c-product__body p').text.strip()

    # Información de precios y stock
    price_info = soup.select_one('.c-product__price')
    socio_price = price_info.select('p.o-text--price-store')[0].text
    normal_price = price_info.select('p.o-text--small')[0].text
    stock = price_info.select('p.o-text--small')[1].text

    # Acceder a la URL de la imagen
    product_image = soup.select_one('div.c-product__thumb img')['src']

    # Crear un diccionario con la información
    product_data = {
        "URL": url,
        "ID del producto": product_id,
        "SKU del producto": product_sku,
        "Nombre del producto": product_name,
        "Marca del producto": product_brand,
        "Categoría del producto": product_category,
        "Variante del producto": product_variant,
        "Precio del producto": product_price,
        "Puntaje de La Cav": lacav_score,
        "Descripción": description,
        "Precio Socio": socio_price,
        "Precio Normal": normal_price,
        "Stock": stock,
        "URL de la imagen": product_image,
        "Viña": wine_data["Viña"],
        "Cepa": wine_data["Cepa"],
        "Cosecha": wine_data["Cosecha"],
        "Procedencia": wine_data["Procedencia"],
        "Línea": wine_data["Línea"],
        "Formato": wine_data["Formato"],
        "Tipo": wine_data["Tipo"],
        "Alcohol": wine_data["Alcohol"],
        "Mezcla": wine_data["Mezcla"],
        "Guarda": wine_data["Guarda"],
        "Maridaje": wine_data["Maridaje"]
    }

    all_product_data.append(product_data)

# Resto del código sin cambios

fieldnames = [
    "URL",
    "ID del producto",
    "SKU del producto",
    "Nombre del producto",
    "Marca del producto",
    "Categoría del producto",
    "Variante del producto",
    "Precio del producto",
    "Puntaje de La Cav",
    "Descripción",
    "Precio Socio",
    "Precio Normal",
    "Stock",
    "URL de la imagen",
    "Viña",
    "Cepa",
    "Cosecha",
    "Procedencia",
    "Línea",
    "Formato",
    "Tipo",
    "Alcohol",
    "Mezcla",
    "Guarda",
    "Maridaje",
    "Cuerpo",
    "Frescor",
    "Dulzura",
    "Astringencia"
]

# Guardar los datos en un archivo CSV
csv_filename = "productos.csv"

with open(csv_filename, mode='w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for product_data in all_product_data:
        writer.writerow(product_data)
